## Sesión 4

En esta última sesión se propone realizar un ejercicio libre con el objetivo de mejorar lo máximo posible el acierto en la tarea MNIST. A modo de orientación se entiende por una buena tasa de acierto en esta tarea cuando obtenemos más de un 99% de acierto.

Para conseguir este resultado podréis probar pipelines que preprocesen los datos adecuadamente. Probar diferentes clasificadores e incluso combinación de los mismos.

In [12]:
## imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# ...
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)


/usr/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

In [3]:
# Partición de Datos
tam_test = 0.1
tam_val = 0.1
tam_train = 0.8

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=tam_test, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=tam_val/(tam_train + tam_test), random_state=23) 

Ejercicio: probar todo lo necesario para obtener un acc > 99%

In [11]:
## Ejercicio obtener acc>99% 
clf1 = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1)
clf2 = HistGradientBoostingClassifier()
clf3 = VotingClassifier(estimators=[('kn', clf1), ('hg', clf2)], n_jobs=-1)
for nc in [4,8,16,32]:
    clf = make_pipeline(PCA(n_components=nc), clf3)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))


La precisión de Pipeline(steps=[('pca', PCA(n_components=4)),
                ('votingclassifier',
                 VotingClassifier(estimators=[('kn',
                                               KNeighborsClassifier(n_jobs=-1,
                                                                    n_neighbors=3,
                                                                    p=1)),
                                              ('hg',
                                               HistGradientBoostingClassifier())],
                                  n_jobs=-1))]) es 61.4%
La precisión de Pipeline(steps=[('pca', PCA(n_components=8)),
                ('votingclassifier',
                 VotingClassifier(estimators=[('kn',
                                               KNeighborsClassifier(n_jobs=-1,
                                                                    n_neighbors=3,
                                                                    p=1)),
                             

In [19]:
clf1 = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1)
clf1bag = BaggingClassifier(clf1, max_samples=0.5, max_features=0.5)
clf2 = HistGradientBoostingClassifier(max_depth=128, max_bins=8)
clf2_2 = make_pipeline(PCA(n_components=32), HistGradientBoostingClassifier(max_depth=128, max_bins=8))
clf3 = VotingClassifier(estimators=[('kn', clf1), ('bgkn', clf1bag), ('hg', clf2), ('pcahg', clf2_2)], voting='soft' ,n_jobs=-1)
#for nc in [4,8,16,32]:
clf = make_pipeline(PCA(n_components=32), clf3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('votingclassifier',
                 VotingClassifier(estimators=[('kn',
                                               KNeighborsClassifier(n_jobs=-1,
                                                                    n_neighbors=3,
                                                                    p=1)),
                                              ('bgkn',
                                               BaggingClassifier(estimator=KNeighborsClassifier(n_jobs=-1,
                                                                                                n_neighbors=3,
                                                                                                p=1),
                                                                 max_features=0.5,
                                                                 max_samples=0.5)),
                                              ('hg',
               